#Support Vector Binary Classification of Disease State with Cycle Averaged Sensor Data
##Input Data: 15 sensor map of the cardiac cycle over a 1 second period ($15\times2000$)

##Investigation of:
* Classifier prediction performance.
* Out of sample prediction error.
* The effect of kernel choice: Linear, Polynomial, RBF.
* The effect of subsampling the time series. From 2000 down to $\sim$200 in steps.

**Daniel Wilson and John Mooney**

In [2]:
import mlpy.wavelet
import mcg
import numpy as np
import datetime
import timeit
import time
import copy
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import time
import gc
from matplotlib.mlab import PCA
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
from sklearn import svm
#matplotlib

mcg.deviceID = 'mk1'
mcg.setDeviceID()

#%matplotlib inline
from skimage.feature import match_template

alphaPlus
mk1


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


#Load data from HDF5 store

In [3]:
#loading in data
store = pd.HDFStore('store.h5')
store

<class 'pandas.io.pytables.HDFStore'>
File path: store.h5
/DF_autosel              frame        (shape->[699,110])
/DF_expertsel            frame        (shape->[428,113])

In [4]:
dfe = store.get('DF_expertsel')

In [5]:
#These are the keys for all the people
TRkeys = ['b3b91a075db3855e2d8d865ea25f3879', '24958c6d339683c981fa4589c611c761',
          'e49a899e4a35586c9cf0abe3b60fac89', 'c77b360b908b84736af82f1c58fffe99',
          'eb71f5928da7ae870825b32707b365d1', 'e29b0a9a0d76396f9825cca9adbd642e',
          '521a2a7408924abe00ade1bd9a3e907c', '685777c5ac77a96bbf4155eb7c2d1dd5',
          'c72c74844c9ae52225b8930bf88c0580', 'a11d66b95134c13c3de8869f4e811582',
          '9aa5263db2a1c22ed5e86db021f7bc0b', '587d11646a73de6b55472af5bf703de7',
          '25b5f2c00541e1a3a3f1eaaf2504f8d2', 'ce82758d2ac42a2f2a197e0b4a3ae04e',
          '3c69dc4d60898ad2632a058d5e11db45', '83d312098265e0e0e09736d447aa734c',
          'eba6cd21897014cd9c672da68f839bef', '9a64265b91bb7cc38ab00b726cbbc725',
          'f64913c0236b1ef9e936e59adcc74fe1', 'fd87f665b8bfed4b0eb5b3850cec4d7f',
          'cce612a6e43e82f3c4bd63d8375ebca7', '2c344015c5c8fca5b1d3162e86b16576',
          'f838df23f40d901f178d22f0230ca96d', '24d12f02353bf3d209946ded6f55ab4e',
          'aa4e2a8dc1f520307b0edc94df4d4883', '027fcab4d250327f001b1e8039e6d9f8',
          '79cfdf26b35c1ed684308488897245b4', '7f0488175e10164d95a4ae0e57c225db',
          '08951c7805a2461962b0d724c864380d', '7d73c33f57b171eb7c7139975b3a6db4',
          '817db5b9bdcff7ae1da53c6ed79baa5a', 'a2d0ca6fb2a5d73944b4a11560fa2bc5',
          '1b5f1203109e2476f5e7e0bb0805714c', '2148dd132138e10f006d43726cda7ed9',
          '20a9eafcd6cf54b494808df3239bde9b', 'e731f7f04bb68ac8a54c7ae4603645ba',
          '9728a01e0e4c0770bea329f4c7dfbd77', '7715a5c99e9e1b8f617980345fed15cf',
          'ecc1ccaebf76887ae5d1fc1f02397e72', '5125afeca6031f96b0b91016057d9526',
          '737acd87d493481fedc5b29e05f9200d', 'a250e177f138e0da5393ec7b15de8cd9',
          '07d1a9b98501db44c41098f1f0897841', '5c2844ef19a0974848fd6b72281da9aa',
          '06cf128ddc7d62bc3cf848eae79b240c', 'bb6a31389d79503f74121491ba9d33cf',
          '1b789758c4f5f7f1534923e8121c832e', 'e8814fa36a4f3ef138331ae4d408035c',
          '20709c4e9e80fef2be632638bb6c066f', 'a6c1776beb8a26272b178f3b278ccb4b',
          'fddcdfa9880b44fe5dc489cbdb0b8e75', 'a0d4a1570f23b604bc851c2efb320859',
          '060276246a2ffd4db1e218a26b850479', '8bcf0e840a586639e5f9d9fdbb7a6631',
          '7185dedbb8c7f280eb6ad3eb635ac5cc', 'd8121c5045b76b09819824cd51df50da',
          'e0ce6ce4220b14f19885966b9b2c4c07', '317b39e1319be27c9410fb87aa6cfe9b',
          '769202eaf703a273f2107f6d6e262ae7', '8f273b9acf5150e07112f9bd43b2cd9a',
          '5f96443a9da689b201478d4d10f13b4d', '52dfb2806ba303139c052780db6563ad',
          '95235e40f223d36e1e8ce58517c7ec65', 'a14103072bb438fe503910ae30a9ec19',
          'e15cb371b7e2d30057895031583e8a3e', '18a77e78f8616d5ce150a97ba0c631b9',
          '1c9dc800cefdb4faa1332fa13a10e38e', '3c5da0d5ce1f76a37ff203f8b12b00da',
          '831405ff01b7d7105a6df86eb104a729', '8ca1948d95f529f6ab5e0b3605959ade',
          'ba485f7d407c75a9849f6758bc8a17c7', 'a0240f2c56f1224e83f7c7f539508864',
          'c4bc20dd85e352af35400d5e9219633d', '1a5b03ea8b6450f584eb1c3194b5a637',
          '4ab0f6d624cc02d9ca8427d35ae96c03', 'bc15e2ce44b19e3947ba83ede74e2f49',
          '56145ce4fae8654cae3dd41bdd12d9c6', 'b56ed7ee8a39b644a3ae427bfad0003a',
          '540c956a76280e5487c98fd35690192b', '135ab1432f211ffa3e77f5a1449c4163',
          'ef9a1981ddef5857b3851ced0f6a9149', '9af089e682f5f7fa445429dfae86b799',
          '48b9a4bfeb80d0bf1cfa8ba96579e82f', '169223512d1fbe477709bcde7143b66a',
          '1bf5617e7ad39a796d55cd4a0edb246f', '3c4034212ebccd7082d161f4f8b5cf6c',
          '413ed3450a56c5b702104034c3e07bbf', 'be194126761c267973f87e620bdca773',
          '0ff5c43bb153fa729aa595a5656f3b4b', '917bc2a8c291422934f40b8990242275',
          'bb1bcb2f97e4ceef38a51973a98b86e0', 'd5b61ad44c964645dedab2dbba550044',
          '343863dcd24c8598c11e447e9a1c15ec', '497f9581a64d7b1b58bb0b4d5759d101',
          '5d9aa192c26f58092490fd75195e49e2', 'da4511230a57b98cb1bf25e970d3e477',
          '1cd806ef4b3b75d33a936649f5089169', '10f2739d3aeed8f388038fade3b9601d',
          '2f4a75bb446a434b77002cccf44ef75d', 'ebd1fe223e567baf5755d7c8595e3987',
          'f3456532816d3ab6241bf04f03d220d8', '9fbdc71e6c0b05998c5ff268e5070b60',
          'd0cb9b0697ed5dd6fe1401297ef2e5f7', '80e1b2a4a0aa33476f5c77b55cc0eb7e',
          'd13a92d7ec271c8b44e9ab33ffef2d23', 'b49ba1022e59137850f21d0b05a0850f',
          '0bb6000451b6e79d88f53636b516f8e3', '4ac21d7b0967f2cc3ad758036f0b5bae',
          'e335900ed09ecb5d54df86f5228549b4', '2759bb03ff319700c3777f05762fa1c1',
          '06ed1b074239a3da07f82d92c31007f2', '520d89bf849014f1cc5b3c27f0af4e3f',
          'ff768828776a35332b1bda7a2c167974', 'be83bf261bb18d8c9fa404ee476d6c5c',
          '61b40a85eb3d9f048ce351f05cc40306', '06906c5292121bfa352fef7ff52d9065',
          '75f27474d4a4c77d606b464f2eb777aa', '011887e97eed8b6cf100eb88fb0b4e55',
          '3a2d15e6d61efaba7b376395b8dcdfc6', '484e6f9ec2fb24005af2e32ce12f51e2',
          '766e2ad3e33166a209b06d48c7eda47c', '97f49ed9622b100b03d80b14ed4011d8',
          'f39ea38c29edcec527a1c82749ee49c5', '79b370935c0ba511ba091d3f6e76ef34',
          '8f9fc69fe3c966cecc0320c357801adf', '3f809554c0c5a953f57ce49ad28b70af', 
          '720bbd6b3d42487dcbde668f73406e27', 'ffe4b39de83a0418698aca54618f4af3']

In [8]:
pathYHC = '/media/john/SAMSUNG/MCG-Scans/50HV-smallCoils/scanDataFrame.pkl'
YHC  = pd.read_pickle(pathYHC)
keysYHC = mcg.keylist_50HV
YHC = YHC[YHC.CoilData != 'a']

/usr/local/lib/python2.7/dist-packages/pandas/core/ops.py:648: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = lib.scalar_compare(x, y, op)


In [11]:
YHC['class']='H'

In [ ]:
path20  = '/media/john/SAMSUNG/MCG-Scans/0-QuantumImaging/scanDataFrame.pkl'


In [13]:
#separtaing out the H's from T's
H = dfe.ix[TRkeys].ix[(dfe['class']=='H')]
T = dfe.ix[TRkeys].ix[(dfe['class']=='T')]
P = dfe.ix[(dfe['class']=='P')]
P['class']='T'
lst = pd.concat([YHC,H,T,P])

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
print len(P)
print len(T)
print len(H)

150
30
98


In [15]:
#DROP THAT NAN - removing nan values from the data
lst = lst.dropna(subset = ['CoilData'])


In [16]:
#Confirming that the Nans have been dropped
lst['CoilData']

036363d2ae08511247bf4def0f47eeb1    [[-3.37185205976e-07, -3.5160176969e-07, -3.64...
049e83cc2bf423be492076f09c6a854a    [[2.24604539628e-07, 2.23998378566e-07, 2.2321...
062a32aa0cc72f503fe30ad7f05f8d2a    [[-2.75507032823e-06, -2.71012780926e-06, -2.6...
06664dce661ec3ab3841846eced60838    [[6.0268599239e-07, 5.91404518158e-07, 5.80276...
06adcae7081b98c39874088c731a54a7    [[2.23792585468e-07, 2.25073230763e-07, 2.2618...
076da7107f6ea74339790fedfd83510f    [[1.64395662832e-07, 1.50202092042e-07, 1.3680...
08fd9e6ebee242550c3f56da0bc1740b    [[2.23792585468e-07, 2.25073230763e-07, 2.2618...
0906359d8a019feb19a1181381657e34    [[-5.59655559954e-07, -5.71003507485e-07, -5.8...
09c572055a89a388c6f5424f38d6e063    [[5.883217431e-07, 5.84777809715e-07, 5.812587...
0b7d5571ee37910a6f2fc81d2f374028    [[2.3412805253e-07, 2.23348873823e-07, 2.12731...
0de2479282740252e7b1a0f3391ca922    [[5.56575038379e-08, 5.20013897299e-08, 4.8510...
11a7839a82f0f9a627701575b36dd470    [[2.23792585468e-0

In [17]:
#splicing H data
print type(H.ix[0,'CoilData'])
print H.ix[0,'CoilData'].shape
print H.ix[0,'CoilData'].T[::2].T.shape
H2 = H.ix[0,'CoilData'].T[::2].T


<type 'numpy.ndarray'>
(15, 3000)
(15, 1500)


In [41]:
#creating a list of classes 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = lst['class'].tolist()
le.fit(y)
y = le.transform(y)
print y.shape

(311,)


In [42]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [43]:
#checking to see if there are any floats, caused by the Nans, should still have 120
X = lst['CoilData'].values.tolist()
for i, x in enumerate(X):
    #print i,type(x)
    if type(x)==float:
        
        print x
        index = []
        index.extend([i])
        #print index
        #X = np.delete(X,index)
print len(X)


311


In [44]:
#Cutting the readings down to 2000
for i, x in enumerate(X):
    #print 'a', X[i].shape
    X[i] = x.T[0:2000].T
    #print 'b', X[i].shape

In [45]:
#flattening X
for i, x in enumerate(X):
    X[i]= x.flatten()

#SVC

In [46]:
#normalising the data, set 0 as mean value
val = preprocessing.scale(X) #CoilData

In [96]:
#val = val.reshape(120,1) #If one feature used need to reshape.

In [47]:
print val.shape

(311, 30000)


In [48]:
clf = svm.SVC(kernel="rbf", probability = True, cache_size = 600, class_weight = 'balanced')


In [49]:
from sklearn.grid_search import GridSearchCV #optimizing gamma and c

gamma_range = np.logspace(-5, -3, 5)
c_range = np.linspace(5, 50, 5)
param_grid = dict(gamma = gamma_range, C = c_range)
print param_grid


{'C': array([  5.  ,  16.25,  27.5 ,  38.75,  50.  ]), 'gamma': array([  1.00000000e-05,   3.16227766e-05,   1.00000000e-04,
         3.16227766e-04,   1.00000000e-03])}


In [50]:
from sklearn import cross_validation
CV = cross_validation.StratifiedKFold(y, n_folds=6, shuffle = True)

In [51]:
grid = GridSearchCV(clf, param_grid, cv=CV, scoring="accuracy")

In [52]:
%time grid.fit(val, y)

CPU times: user 20min 4s, sys: 5.95 s, total: 20min 10s
Wall time: 20min 10s


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 0 ..., 1 1], n_folds=6, shuffle=True, random_state=None),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=600, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  5.  ,  16.25,  27.5 ,  38.75,  50.  ]), 'gamma': array([  1.00000e-05,   3.16228e-05,   1.00000e-04,   3.16228e-04,
         1.00000e-03])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [53]:
print grid.best_score_

0.855305466238


In [104]:
#grid.grid_scores_

In [106]:
#Draws validation accuracy heatmap
from matplotlib.colors import Normalize
scores = [x[1] for x in grid.grid_scores_]
scores = np.array(scores).reshape(len(c_range), len(gamma_range))

class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))


plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.81))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=90)
plt.yticks(np.arange(len(c_range)), c_range)
plt.title('Validation accuracy')
plt.show()

In [45]:
print grid.best_score_
print grid.best_params_
#print grid.grid_scores_[79][1]

NameError: name 'grid' is not defined

In [33]:
#optimum clf here,with the best params
#clf = svm.SVC(kernel="rbf" , probability = True, cache_size = 600, class_weight = "balanced" ,
#              C = grid.best_params_['C'], gamma = grid.best_params_['gamma'])

#{'C': 10.0, 'gamma': 0.0001}

clf = svm.SVC(kernel="rbf" , probability = True, cache_size = 600, class_weight = "balanced",
              C = 10, gamma = 0.0001)

In [84]:
#kfold cross validation

#shuffle needed otherwise test sets are all healthy or unhealthy
#for train, test in kf_total:
    #print train, '\n', test, '\n\n'

In [31]:
#cross validation score
CVS = cross_validation.cross_val_score
scores = CVS(clf, val, y, cv=CV, n_jobs = 1)
print scores

[ 0.90625     0.93548387  0.80645161  0.77419355  0.83870968  0.90322581
  0.93548387  0.83870968  0.74193548  0.83870968]


In [86]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.82 (+/- 0.11)


In [87]:
#this tells us how well its actualy performing, due to the unbalanced data
from sklearn.metrics import confusion_matrix

sen_list = []
spec_list = []


for i, (train, test) in enumerate(CV):
    y_pred = clf.fit(val[train], y[train]).predict(val[test])
    #print y_pred,"predicted"
    #print y[test], "actual"
    cm = confusion_matrix(y[test], y_pred)

    tn = float(cm[0][0])/np.sum(cm[0])
    tp = float(cm[1][1])/np.sum(cm[1])
    sen_list.append(tp)
    spec_list.append(tn)

mean_sen = sum(sen_list)/float(len(sen_list))
mean_spec = sum(spec_list)/float(len(sen_list))
print "Mean sensitivity is", mean_sen
print "Mean specificity is", mean_spec

Mean sensitivity is 0.183333333333
Mean specificity is 0.978888888889


In [88]:
SV = clf.n_support_
print sum(SV),SV



102 [80 22]


##Useful Functions

In [89]:
print val.shape
print grid.best_params_

(120L, 8L)
{'C': 1.0, 'gamma': 1.0}


In [37]:
from sklearn.metrics import roc_curve, auc
from scipy import interp

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (train, test) in enumerate(CV):
    probas_ = clf.fit(val[train], y[train]).predict_proba(val[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')

print auc(fpr, tpr)
mean_tpr /= len(CV)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC - Sens = ' + str(np.around(mean_sen, 2)) + " Spec = " + str(np.around(mean_spec, 2))
          + " $\gamma$ = " + str(grid.best_params_["gamma"]) + " C = " + str(grid.best_params_["C"]) + " #SV = " + str(sum(SV)))
plt.legend(loc='best', bbox_to_anchor=(0.5, 1.05))
plt.show()



ValueError: class_weight must be dict, 'auto', or None, got: 'balanced'

In [ ]:
%matplotlib

In [39]:
from sklearn import cross_validation
#CV = cross_validation.ShuffleSplit(len(y), n_iter=20,
#                                   test_size=0.1, random_state=0)
CV = cross_validation.StratifiedKFold(y, n_folds=3, shuffle = True)

from sklearn.learning_curve import learning_curve
for i, (train, test) in enumerate(CV):
    y_pred = clf.fit(val[train], y[train]).predict(val[test])
    print y[test], "actual"

#range(65,80)
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=CV,
                        n_jobs=1, train_sizes=range(65,171)):

    
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, n_jobs=n_jobs, cv=CV, train_sizes=train_sizes, verbose=10)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    print train_sizes
    return plt


title = "Learning Curves (SVM, RBF kernel)"




plot_learning_curve(clf, title, val, y, cv = CV, n_jobs=1)

plt.show()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[learning_curve] Training set sizes: [ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82
  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140

ValueError: The number of classes has to be greater than one; got 1

In [38]:
from sklearn import cross_validation
#CV = cross_validation.ShuffleSplit(len(y), n_iter=20,
#                                   test_size=0.1, random_state=0)
CV = cross_validation.StratifiedKFold(y, n_folds=3, shuffle = True)

from sklearn.learning_curve import learning_curve
for i, (train, test) in enumerate(CV):
    y_pred = clf.fit(val[train], y[train]).predict(val[test])
    print y[test], "actual"

#range(65,80)
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=CV,
                        n_jobs=1, train_sizes=range(65,100)):

    
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, n_jobs=n_jobs, cv=CV, train_sizes=train_sizes, verbose=10)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    print train_sizes
    return plt


title = "Learning Curves (SVM, RBF kernel)"




plot_learning_curve(clf, title, val, y, cv = CV, n_jobs=1)

plt.show()



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] actual
[learning_curve] Training set sizes: [65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89
 90 91 92 93 94 95 96 97 98 99]
[CV] no parameters to be set .........................................


ValueError: The number of classes has to be greater than one; got 1

In [108]:
print y

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]


In [151]:


def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(y[names]))
    plt.xticks(tick_marks, y[names], rotation=45)
    plt.yticks(tick_marks, y_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_pred = clf.fit(val[train], y[train]).predict(val[test])
    
# Compute confusion matrix
cm = confusion_matrix(y[test], y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[10  0]
 [ 2  0]]


NameError: global name 'names' is not defined

In later work it will be interesting to reduce the dimensionality of the input. In theory this will reduce the classifiers out of sample prediction error (i.e. predicting on data not in the training set).  

This can be done by feature extraction. Resulting in a timeseries of dipole properties (Angle, Length, MinMaxRatio, Dipole Center position and a measure of each peaks circular symmetry (perhaps using a 2D correlation with an idealised dipole of equivalent angle and length) or with an angluar variance measure?) 

In [28]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.learning_curve import learning_curve

In [29]:
digits = load_digits()
X, y = digits.data, digits.target

In [ ]:
y